MOVIE RECOMMENDATION SYSTEM 

CONTENT BASED RECOMMENDATION SYSTEM USING COSINE SIMILARITY

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
movies = pd.read_csv(r"C:\Users\DELL\Documents\AIML SOURCE\PROJECT\IMDB MOVIES DATA.csv")

In [4]:
movies.head(1)

,id,title,vote_average,vote_count,release_date,revenue,runtime,adult,backdrop_path,budget,...,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,8.364,34495,7/15/2010,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"


In [5]:
movies.shape

(14999, 22)

In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    14999 non-null  int64  
 1   title                 14999 non-null  object 
 2   vote_average          14999 non-null  float64
 3   vote_count            14999 non-null  int64  
 4   release_date          14999 non-null  object 
 5   revenue               14999 non-null  int64  
 6   runtime               14999 non-null  int64  
 7   adult                 14999 non-null  bool   
 8   backdrop_path         14963 non-null  object 
 9   budget                14999 non-null  int64  
 10  homepage              5325 non-null   object 
 11  imdb_id               14983 non-null  object 
 12  original_language     14999 non-null  object 
 13  original_title        14999 non-null  object 
 14  overview              14975 non-null  object 
 15  popularity         

Preprocessing

In [7]:
movies = movies[['id','title','release_date','revenue','budget','genres','popularity','original_language','overview']]

In [8]:
movies.head()

,id,title,release_date,revenue,budget,genres,popularity,original_language,overview
0,27205,Inception,7/15/2010,825532764,160000000,"Action, Science Fiction, Adventure",83.952,en,"Cobb, a skilled thief who commits corporate es..."
1,157336,Interstellar,11/5/2014,701729206,165000000,"Adventure, Drama, Science Fiction",140.241,en,The adventures of a group of explorers who mak...
2,155,The Dark Knight,7/16/2008,1004558444,185000000,"Drama, Action, Crime, Thriller",130.643,en,Batman raises the stakes in his war on crime. ...
3,19995,Avatar,12/15/2009,2923706026,237000000,"Action, Adventure, Fantasy, Science Fiction",79.932,en,"In the 22nd century, a paraplegic Marine is di..."
4,24428,The Avengers,4/25/2012,1518815515,220000000,"Science Fiction, Action, Adventure",98.082,en,When an unexpected enemy emerges and threatens...


In [9]:
movies.shape

(14999, 9)

In [10]:
#check for empty cells
movies.isnull().sum()

id                    0
title                 0
release_date          0
revenue               0
budget                0
genres               10
popularity            0
original_language     0
overview             24
dtype: int64

In [11]:
#removed all empty values
movies.dropna(inplace=True)

In [12]:
movies.duplicated().sum()

0

In [13]:
movies['overview'][0]
#it's a str.

'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.'

In [14]:
#to convert it in list format
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [15]:
movies['genres']

0                 Action, Science Fiction, Adventure
1                  Adventure, Drama, Science Fiction
2                     Drama, Action, Crime, Thriller
3        Action, Adventure, Fantasy, Science Fiction
4                 Science Fiction, Action, Adventure
                            ...                     
14994                                  Drama, Horror
14995                                 Action, Comedy
14996                      Animation, Music, Fantasy
14997                                         Comedy
14998                                Drama, Thriller
Name: genres, Length: 14965, dtype: object

In [16]:
movies['genres'] = movies['genres'].apply(lambda x:x.split(','))

In [18]:
#removed space between words to make it singl entity
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])


In [20]:
#creating new column 
movies['tags'] = movies['genres'] + movies['overview']

In [21]:
movies.head()

,id,title,release_date,revenue,budget,genres,popularity,original_language,overview,tags
0,27205,Inception,7/15/2010,825532764,160000000,"[Action, ScienceFiction, Adventure]",83.952,en,"[Cobb,, a, skilled, thief, who, commits, corpo...","[Action, ScienceFiction, Adventure, Cobb,, a, ..."
1,157336,Interstellar,11/5/2014,701729206,165000000,"[Adventure, Drama, ScienceFiction]",140.241,en,"[The, adventures, of, a, group, of, explorers,...","[Adventure, Drama, ScienceFiction, The, advent..."
2,155,The Dark Knight,7/16/2008,1004558444,185000000,"[Drama, Action, Crime, Thriller]",130.643,en,"[Batman, raises, the, stakes, in, his, war, on...","[Drama, Action, Crime, Thriller, Batman, raise..."
3,19995,Avatar,12/15/2009,2923706026,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",79.932,en,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction, I..."
4,24428,The Avengers,4/25/2012,1518815515,220000000,"[ScienceFiction, Action, Adventure]",98.082,en,"[When, an, unexpected, enemy, emerges, and, th...","[ScienceFiction, Action, Adventure, When, an, ..."


In [22]:
movies.shape

(14965, 10)

In [23]:
new_df = movies[['id','title','tags']]

In [24]:
new_df

,id,title,tags
0,27205,Inception,"[Action, ScienceFiction, Adventure, Cobb,, a, ..."
1,157336,Interstellar,"[Adventure, Drama, ScienceFiction, The, advent..."
2,155,The Dark Knight,"[Drama, Action, Crime, Thriller, Batman, raise..."
3,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."
4,24428,The Avengers,"[ScienceFiction, Action, Adventure, When, an, ..."
...,...,...,...
14994,397514,The Windmill Massacre,"[Drama, Horror, Jennifer,, an, Australian, gir..."
14995,31000,Amos & Andrew,"[Action, Comedy, When, Andrew, Sterling,, a, s..."
14996,124277,The Maker,"[Animation, Music, Fantasy, A, strange, creatu..."
14997,869641,Vacation Friends 2,"[Comedy, Newly, married, couple, Marcus, and, ..."


In [25]:
#just converted list to str and inserted in tags itself
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\DELL\AppData\Local\Temp\ipykernel_8984\1800306622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [26]:
new_df.head()

,id,title,tags
0,27205,Inception,"Action ScienceFiction Adventure Cobb, a skille..."
1,157336,Interstellar,Adventure Drama ScienceFiction The adventures ...
2,155,The Dark Knight,Drama Action Crime Thriller Batman raises the ...
3,19995,Avatar,Action Adventure Fantasy ScienceFiction In the...
4,24428,The Avengers,ScienceFiction Action Adventure When an unexpe...


In [27]:
import nltk

In [28]:
#using stemming to make data to be more easily readable for model
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer() #stemming algorithm

In [29]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [30]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8984\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [31]:
new_df['tags'][0]

'action sciencefict adventur cobb, a skill thief who commit corpor espionag by infiltr the subconsci of hi target is offer a chanc to regain hi old life as payment for a task consid to be impossible: "inception", the implant of anoth person\' idea into a target\' subconscious.'

In [32]:
#best to convert these tags into lower case 
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\DELL\AppData\Local\Temp\ipykernel_8984\1343125587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [33]:
new_df.head()

,id,title,tags
0,27205,Inception,"action sciencefict adventur cobb, a skill thie..."
1,157336,Interstellar,adventur drama sciencefict the adventur of a g...
2,155,The Dark Knight,drama action crime thriller batman rais the st...
3,19995,Avatar,action adventur fantasi sciencefict in the 22n...
4,24428,The Avengers,sciencefict action adventur when an unexpect e...


Vectorization

text vecetorization using bag of words

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

In [35]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [36]:
cv.fit_transform(new_df['tags']).toarray().shape

(14965, 10000)

In [37]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [38]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zones', 'zoo', 'zoro'], dtype=object)

stemming

In [39]:
ps.stem('actions')

'action'

In [40]:
stem('Action ScienceFiction Adventure Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.')

'action sciencefict adventur cobb, a skill thief who commit corpor espionag by infiltr the subconsci of hi target is offer a chanc to regain hi old life as payment for a task consid to be impossible: "inception", the implant of anoth person\' idea into a target\' subconscious.'

calculating distance b/w movie vectors using cosine_similarity

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
similarity = cosine_similarity(vectors)

In [43]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(404, 0.3592106040535498),
 (887, 0.35560035560053344),
 (4075, 0.3519947346980235),
 (8941, 0.3486948369707026),
 (5228, 0.33601075251612356),
 (2582, 0.33168807323853644),
 (11145, 0.33020033020049533),
 (837, 0.32995600879804493),
 (11044, 0.32128773156099966),
 (2150, 0.32128773156099955)]

MAIN FUNCTION

In [44]:
'''
creating a function to print the movie names
'''

'\ncreating a function to print the movie names\n'

In [45]:
def recommand(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]

    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        

In [54]:
recommand('Coco')

Let It Shine
The Book of Life
Tarzan II
Dragon Quest: Your Story
Kiwi!
Rock Dog
Abominable
Mo' Better Blues
Kind Hearts and Coronets
Pinocchio


In [47]:
import pickle

In [48]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [49]:
new_df['title'].values

array(['Inception', 'Interstellar', 'The Dark Knight', ..., 'The Maker',
       'Vacation Friends 2', 'The Monitor'], dtype=object)

In [50]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [51]:
pickle.dump(similarity,open('similarity.pkl','wb'))